In [1]:
from re import S
from fastapi.params import Query
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from dotenv import load_dotenv
import os, sys, json, time

In [14]:
def update_music_info_in_sheets():
    """
    Updates the text in cell H1 to 'music info' for each user's Google Sheet.
    """
    with open("db/main/db.json", "r", encoding="utf-8") as file:
        table_list = json.load(file)

    for index, account in enumerate(table_list["accounts"]):
        if index != 0:
            try:
                # Set up Google Sheets API credentials
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive"
                ]
                credentials = ServiceAccountCredentials.from_json_keyfile_name(
                    'static/reelstranscription-a94a4b07252e.json', scope)
                client = gspread.authorize(credentials)

                # Open the Google Sheet
                google_sheet_url = f'https://docs.google.com/spreadsheets/d/{account["table_id"]}/edit?usp=sharing'
                sheet = client.open_by_url(google_sheet_url).worksheet('INSTAGRAM')

                # Read and print the current value of cell H1
                # current_value = sheet.acell('H1').value
                # print(f"Current value in H1 for {index} -  {account['username']}: {current_value}")

                # Update cell H1

                sheet.batch_update([
                    {"range": "H1", "values": [[""]]},
                    {"range": "U1", "values": [["music info"]]}
                ])
                print(f"Updated H1 for {account['username']}")
                time.sleep(2)

            except Exception as e:
                print(f"Ошибка в процессе обновления H1 для {account['username']}: {e}")

update_music_info_in_sheets()

Updated H1 for dashatripls
Updated H1 for dr.chshtnv
Updated H1 for a.saubanovaa
Updated H1 for mellediaaa
Updated H1 for elena_shkul
Updated H1 for a_mladinov
Updated H1 for kseniiagallianova
Updated H1 for rbuddy
Ошибка в процессе обновления H1 для solokumi: APIError: [403]: The caller does not have permission
Updated H1 for vartsergey
Updated H1 for daria_koziakova
Updated H1 for potok_sociala
Updated H1 for potok_lifeplus
Updated H1 for potok_theexpertgarden
Updated H1 for potok_wifimap
Updated H1 for potok_5prizm
Updated H1 for potok_progkids
Updated H1 for potok_raqobatchilar
Updated H1 for potok_germany
Updated H1 for potok_kumar&solo
Updated H1 for potok_kumi
Updated H1 for potok_damir
Updated H1 for potok_olya_makarova
Updated H1 for potok_maria_ivakova
Updated H1 for potok_diana
Updated H1 for potok_vkovalevskiy
Updated H1 for potok_itboat
Updated H1 for potok_ahrapov
Updated H1 for sternMeister
